In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
%matplotlib inline

In [ ]:
# Read the training data
dataset = pd.read_csv('Google_Stock_Price_Train')
dataset.head()

In [ ]:
# Locate the ones we want
training = dataset.iloc[:,1:2].values
print(training)
print(training.shape)

In [ ]:
# Scale the data
scaler = MinMaxScaler(feature_range=(1,2))
scaled_training_set = scaler.fit_transform(training)

In [ ]:
# Create data structures
X_train = []
Y_train = []
for i in range(60, 1258):
    X_train.append(scaled_training_set[i - 60:i, 0])
    Y_train.append(scaled_training_set[i, 0])
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
# Reshape the data
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

In [ ]:
# Model with LTSM
regressor = Sequential()

regressor.add(LSTM(units=50, return_sequences=True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

In [ ]:
# Fitting the Model
regressor.compile(optimizer='adam', loss='mean_sqaure_error')
regressor.fit(X_train, Y_train, epochs=100, batch_size=32)

In [ ]:
# Extracting the real prices for comparison
data_test = pd.read_csv('Google_Stock_Price_Test')
actual_price = data_test.iloc[:,1:2].values

In [ ]:
# Preparing inputs for the model
dataset_test = pd.concat((dataset['open'], data_test['open']),axis=0)
inputs = dataset_test[len(dataset_test) - len(dataset_test)-60:].values

inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)

X_test = []
for i in range(60,80):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Make predictions
predicted_price = regressor.predict(X_test)
predicted_price = scaler.inverse_transform(predicted_price)

In [ ]:
# Plot that shit
plt.plot(actual_price, color='red', label='Actual Price')
plt.plot(predicted_price, color='blue', label='Predicted Price')
plt.title('Google Stock Price Predictions')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend